# Imports des bibliothèques

In [19]:
import numpy as np
import pandas as pd
import torch

# Imports des données

In [20]:
data_url = 'data/data_fusion_sentiment.csv'
data = pd.read_csv(data_url)

# Récupérer 10 lignes aléatoires
data_test = data.sample(10)

In [21]:
data_test.shape

(10, 20)

In [22]:
data_test.columns

Index(['Titre', 'Description', 'Auteurs', 'Image', 'Lien Google', 'Editeur',
       'Date publication', 'infoLink', 'Genre', 'Nb scores', 'Id', 'Prix',
       'User_id', 'Nom lecteur', 'revue/utilité', 'revue/score', 'revue/heure',
       'revue/résumé', 'revue/texte', 'Sentiment'],
      dtype='object')

# Prédictions

## Prédictions de sentiments

## Prédictions de notes

### Prédictions BERT

#### Fonctions et setup

In [23]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [24]:
# Fonction de prédictions pour BERT
def predict_review(review, model, tokenizer, device):
    model.eval()
    encoded = tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
    )
    input_id = torch.tensor([encoded["input_ids"]]).to(device)
    attention_mask = torch.tensor([encoded["attention_mask"]]).to(device)
    with torch.no_grad():
        outputs = model(input_id, attention_mask=attention_mask)
    logits = outputs.logits.detach().cpu().numpy()
    predicted_score = logits.argmax(axis=-1)[0] + 1  # Recaler le score pour correspondre à l'échelle originale
    return predicted_score

In [25]:
# Charger le modèle et le tokenizer
model_path = "models\model_A100_sample"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-11): 12 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False

In [27]:
# Prédire les scores des 10 lignes aléatoires
data_test["predicted_score"] = data_test["revue/texte"].apply(lambda x: predict_review(x, model, tokenizer, device))

RuntimeError: Numpy is not available